### Random Forest Classifier

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
%matplotlib inline

In [11]:
df = pd.read_csv('data/train.csv')

#### Feature Selection - First Cut
##### Drop:  PassengerId, Survived , Name, Cabin, Embarked, Ticket
##### Keep: Pclass, Sex, Age, SibSp, Parch, Fare

In [12]:
predict_features = df.drop(['PassengerId','Survived','Name','Ticket','Cabin','Embarked'], axis=1)
predict_features.describe()

,Pclass,Age,SibSp,Parch,Fare
count,891.000000,714.000000,891.000000,891.000000,891.000000
mean,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.420000,0.000000,0.000000,0.000000
25%,2.000000,20.125000,0.000000,0.000000,7.910400
50%,3.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,38.000000,1.000000,0.000000,31.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200


##### Replace missing ages with mean age of population


In [13]:
mean_age = predict_features['Age'].mean(axis=0,skipna=True)
mean_age

29.69911764705882

In [14]:
predict_features.loc[predict_features.Age.isnull(),'Age'] = mean_age
# predict_features

In [15]:
one_hot_training_predictors = pd.get_dummies(predict_features)
# one_hot_training_predictors

In [16]:
y_predict = df['Survived']    

In [17]:
train_X, val_X, train_y, val_y = train_test_split(one_hot_training_predictors, y_predict, random_state = 214)
val_X.shape

(223, 7)

In [18]:
rf_classifier = RandomForestClassifier(random_state=1)
rf_classifier.fit(train_X, train_y)
survivor_preds = rf_classifier.predict(val_X)
survivor_preds

array([1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0], dtype=int64)

In [19]:
print(confusion_matrix(val_y,survivor_preds))
tn, fp, fn, tp = confusion_matrix(val_y,survivor_preds).ravel()
(tn, fp, fn, tp )

[[124  18]
 [ 30  51]]


(124, 18, 30, 51)

In [20]:
f1_score(val_y, survivor_preds)

0.68

In [21]:
print("Recall= ", tp/(tp+fn))
print("Precision= ", tp/(tp+fp))

Recall=  0.6296296296296297
Precision=  0.7391304347826086


#### Try different number of trees

In [22]:
tree_options = [10,30,50,100,200,500,1000,2000,4000]
for trees in tree_options:
    print("Tree count= ", trees)
    rf_classifier = RandomForestClassifier(trees, random_state=1)
    rf_classifier.fit(train_X, train_y)
    survivor_preds = rf_classifier.predict(val_X)
    tn, fp, fn, tp = confusion_matrix(val_y,survivor_preds).ravel()
    print("Accuracy: ", (tn+tp)/(val_X.shape[0]) )

Tree count=  10
Accuracy:  0.7847533632286996
Tree count=  30
Accuracy:  0.7892376681614349
Tree count=  50
Accuracy:  0.7892376681614349
Tree count=  100
Accuracy:  0.8026905829596412
Tree count=  200
Accuracy:  0.8026905829596412
Tree count=  500
Accuracy:  0.8071748878923767
Tree count=  1000
Accuracy:  0.8071748878923767
Tree count=  2000
Accuracy:  0.8071748878923767
Tree count=  4000
Accuracy:  0.8071748878923767


#### Best tree count = 500

In [29]:
rf_classifier = RandomForestClassifier(500, random_state=1)
rf_classifier.fit(train_X, train_y)
survivor_preds = rf_classifier.predict(val_X)
print("Confusion matrix:\n",confusion_matrix(val_y,survivor_preds))
tn, fp, fn, tp = confusion_matrix(val_y,survivor_preds).ravel()
print("Accuracy: ", (tn+tp)/(val_X.shape[0]) )

Confusion matrix:
 [[125  17]
 [ 26  55]]
Accuracy:  0.8071748878923767


In [30]:
print(train_X.describe())
importances = rf_classifier.feature_importances_

indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(train_X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

           Pclass         Age       SibSp       Parch        Fare  Sex_female  \
count  668.000000  668.000000  668.000000  668.000000  668.000000  668.000000   
mean     2.305389   29.540641    0.505988    0.372754   32.600218    0.356287   
std      0.837377   12.629669    1.087599    0.774581   49.286826    0.479260   
min      1.000000    0.420000    0.000000    0.000000    0.000000    0.000000   
25%      2.000000   22.000000    0.000000    0.000000    7.895800    0.000000   
50%      3.000000   29.699118    0.000000    0.000000   14.427100    0.000000   
75%      3.000000   35.000000    1.000000    0.000000   31.620825    1.000000   
max      3.000000   80.000000    8.000000    5.000000  512.329200    1.000000   

         Sex_male  
count  668.000000  
mean     0.643713  
std      0.479260  
min      0.000000  
25%      0.000000  
50%      1.000000  
75%      1.000000  
max      1.000000  
Feature ranking:
1. feature 1 (0.282903)
2. feature 4 (0.277536)
3. feature 5 (0.136354)
4